# EDA

In [21]:
import pandas as pd
import numpy as np
import sweetviz
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import lightgbm as lgb

# titanic data

In [44]:
path = '../../data/titanic.csv'
df = pd.read_csv(path)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [45]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [46]:
# dtypes
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [47]:
# null check
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [48]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

## PassengerId, Name, cabin, Ticket 제거

In [50]:
df = df.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis =1)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


# Missing value
+ age
+ cabin
+ embarked

## IterativeImputer 랜덤포래스트 기반, numerical value만 가능

In [51]:
# explicitly require this experimental feature
# now you can import normally from sklearn.impute
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Load data
# titanic = pd.read_csv("titanic.csv")
titanic = df.loc[:, ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]

# Run imputer with a Random Forest estimator
imp = IterativeImputer(RandomForestRegressor(), max_iter=10, random_state=0)
titanic = pd.DataFrame(imp.fit_transform(titanic), columns=titanic.columns)

/Users/sungchan/opt/anaconda3/envs/eda/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [52]:
titanic.isnull().sum()

Pclass    0
Age       0
SibSp     0
Parch     0
Fare      0
dtype: int64

In [53]:
titanic.head()

,Pclass,Age,SibSp,Parch,Fare
0,3.0,22.0,1.0,0.0,7.2500
1,1.0,38.0,1.0,0.0,71.2833
2,3.0,26.0,0.0,0.0,7.9250
3,1.0,35.0,1.0,0.0,53.1000
4,3.0,35.0,0.0,0.0,8.0500


In [54]:
df = df.drop(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], axis = 1)
df = pd.concat([df, titanic], axis = 1)
df.head()

,Survived,Sex,Embarked,Pclass,Age,SibSp,Parch,Fare
0,0,male,S,3.0,22.0,1.0,0.0,7.2500
1,1,female,C,1.0,38.0,1.0,0.0,71.2833
2,1,female,S,3.0,26.0,0.0,0.0,7.9250
3,1,female,S,1.0,35.0,1.0,0.0,53.1000
4,0,male,S,3.0,35.0,0.0,0.0,8.0500


In [55]:
df.isnull().sum()

Survived    0
Sex         0
Embarked    2
Pclass      0
Age         0
SibSp       0
Parch       0
Fare        0
dtype: int64

# Categorical missing value

In [59]:
# fill most frequent
df_cat = df['Embarked'] 
# df_cat = df_cat.apply(lambda x : x.fillna(x.value_counts().index[0]))
df_cat = df_cat.fillna(df_cat.mode().iloc[0])
df_cat.isnull().sum()

0

In [61]:
df['Embarkde'] = df_cat
df.isnull().sum()

Survived    0
Sex         0
Embarked    2
Pclass      0
Age         0
SibSp       0
Parch       0
Fare        0
Embarkde    0
dtype: int64

# one-hot encoding

# BaseLine model

In [11]:
# X, y split
X = df.drop('Survived', axis=1)
y = df['Survived']

In [12]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
from flaml import AutoML
automl = AutoML()

In [14]:
# 모델 튜닝하기
settings = {
    "time_budget": 60,  # total running time in seconds
    "metric": 'rmse',  # can be: 'r2', 'rmse', 'mae', 'mse', 'accuracy', 'roc_auc', 'roc_auc_ovr',
                           # 'roc_auc_ovo', 'log_loss', 'mape', 'f1', 'ap', 'ndcg', 'micro_f1', 'macro_f1'
    "task": 'classification',  # task type    
    "log_file_name": 'titanic.log',  # flaml log file
    "seed": 421266,    # random seed
}

# 파라미터 참고 링크
# https://microsoft.github.io/FLAML/

In [15]:
automl.fit(X_train=X_train, y_train=y_train, **settings)

[flaml.automl: 11-12 21:26:24] {1485} INFO - Data split method: stratified
[flaml.automl: 11-12 21:26:24] {1489} INFO - Evaluation method: cv
[flaml.automl: 11-12 21:26:24] {1540} INFO - Minimizing error metric: rmse
[flaml.automl: 11-12 21:26:24] {1577} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'lrl1']
[flaml.automl: 11-12 21:26:24] {1826} INFO - iteration 0, current learner lgbm
[flaml.automl: 11-12 21:26:24] {1943} INFO - Estimated sufficient time budget=849s. Estimated necessary time budget=14s.
[flaml.automl: 11-12 21:26:24] {2023} INFO -  at 0.1s,	estimator lgbm's best error=0.4557,	best estimator lgbm's best error=0.4557
[flaml.automl: 11-12 21:26:24] {1826} INFO - iteration 1, current learner lgbm
[flaml.automl: 11-12 21:26:24] {2023} INFO -  at 0.2s,	estimator lgbm's best error=0.4557,	best estimator lgbm's best error=0.4557
[flaml.automl: 11-12 21:26:24] {1826} INFO - iteration 2, current learner lgbm
[flaml.automl: 11-12 21:26:24] {202

# Flaml 결과

In [16]:
''' retrieve best config and best learner'''
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: xgboost
Best hyperparmeter config: {'n_estimators': 27, 'max_leaves': 74, 'min_child_weight': 3.4422292225457083, 'learning_rate': 0.19209004897991425, 'subsample': 0.8117768577596389, 'colsample_bylevel': 0.7329879502932973, 'colsample_bytree': 0.6190181882874791, 'reg_alpha': 0.003954032677116378, 'reg_lambda': 0.20182951689767617}
Best accuracy on validation data: 0.598
Training duration of best run: 0.06905 s


In [17]:
automl.model.estimator

XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.7329879502932973, colsample_bynode=1,
              colsample_bytree=0.6190181882874791, gamma=0, gpu_id=-1,
              grow_policy='lossguide', importance_type='gain',
              interaction_constraints='', learning_rate=0.19209004897991425,
              max_delta_step=0, max_depth=0, max_leaves=74,
              min_child_weight=3.4422292225457083, missing=nan,
              monotone_constraints='()', n_estimators=27, n_jobs=-1,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0.003954032677116378, reg_lambda=0.20182951689767617,
              scale_pos_weight=1, subsample=0.8117768577596389,
              tree_method='hist', use_label_encoder=False,
              validate_parameters=1, verbosity=0)

In [18]:
''' compute predictions of testing dataset ''' 
y_pred = automl.predict(X_test)
print('Predicted labels', y_pred)
print('True labels', y_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]


Predicted labels [1 0 0 1 1 1 1 1 1 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 1 0
 1 1 0 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 1 1
 0 0 1 1 1 1 0 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1
 0 1 1 1 0 0 1 1 0 1 1 1 1 0 0 1 0 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 1 1 0 0
 1 0 1 0 0 1 0 0 0 1 1 1 0 0 0 1 1 0 0 1 0 0 0 1 1 1 0 0 1 1 1]
True labels 709    1
439    0
840    0
720    1
39     1
      ..
433    0
773    0
25     1
84     1
10     1
Name: Survived, Length: 179, dtype: int64


In [19]:
''' compute different metric values on testing dataset'''
from flaml.ml import sklearn_metric_loss_score
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred, y_test))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test))


accuracy = 0.776536312849162
roc_auc = 0.8371943371943372
log_loss = 0.520130294026526


# Flaml 변동성이 좀 있네 테스트 필요 1번의 lgbm 파라미터 그대로 세팅


In [23]:
# Lightgbm
train_data = lgb.Dataset(X_train, label = y_train)
test_data = lgb.Dataset(X_test, label = y_test, reference = train_data)
params = {'n_estimators': 7,
        'num_leaves': 5, 
        'min_child_samples': 21, 
        'learning_rate': 0.32556254798353695, 
        'log_max_bin': 8, 
        'colsample_bytree': 1.0, 
        'reg_alpha': 0.0009765625, 
        'reg_lambda': 1.4786212688867904}

In [25]:
# categorical data 안됨
# one-hot encoding 필요
model = lgb.train(params, train_data, valid_sets = [test_data], num_boost_round = 300,
                early_stopping_rounds= 30,
                valid_names =['valid'])

/Users/sungchan/opt/anaconda3/envs/eda/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/sungchan/opt/anaconda3/envs/eda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: Name, Sex, Ticket, Cabin, Embarked

In [26]:
clf = LGBMClassifier(params)

In [27]:
clf.fit(X_train, y_train)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: Name, Sex, Ticket, Cabin, Embarked

In [ ]:
clf = LGBMClassifier(n_estimators=100,
                     num_leaves=64,
                     max_depth=5,
                     learning_rate=0.1,
                     random_state=1000,
                     n_jobs=-1)
start = time.time()
clf.fit(X_train,y_train)
elapsed = time.time() - start
print(f'LGBM Training ran in {elapsed:.5f} seconds')
y_pred = clf.predict(X_test)
print(f'Test Accuracy: {accuracy_score(y_test,y_pred):.2f}')